In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import math as m

In [2]:
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFE

In [4]:
df = pd.read_csv(r'E:\NEU\ML Ops\Project\StockPricePrediction\data\final_dataset.csv')

c:\Users\kandi\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (7,8,9,10,11,14,15,16,17,18,19,20,21,23,24,25,26,27,28,29,30,31,32,33) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
df

date  Mkt-RF  SMB  HML  RMW  CMA  RF DGS10 RIFSPPFAAD90NB DGS1  \
0      1854-12-01     NaN  NaN  NaN  NaN  NaN NaN   NaN            NaN  NaN   
1      1854-12-02     NaN  NaN  NaN  NaN  NaN NaN   NaN            NaN  NaN   
2      1854-12-03     NaN  NaN  NaN  NaN  NaN NaN   NaN            NaN  NaN   
3      1854-12-04     NaN  NaN  NaN  NaN  NaN NaN   NaN            NaN  NaN   
4      1854-12-05     NaN  NaN  NaN  NaN  NaN NaN   NaN            NaN  NaN   
...           ...     ...  ...  ...  ...  ...  ..   ...            ...  ...   
62051  2024-10-21     NaN  NaN  NaN  NaN  NaN NaN   NaN            NaN  NaN   
62052  2024-10-22     NaN  NaN  NaN  NaN  NaN NaN   NaN            NaN  NaN   
62053  2024-10-23     NaN  NaN  NaN  NaN  NaN NaN   NaN            NaN  NaN   
62054  2024-10-24     NaN  NaN  NaN  NaN  NaN NaN   NaN            NaN  NaN   
62055  2024-10-25     NaN  NaN  NaN  NaN  NaN NaN   NaN            NaN  NaN   

       ... DCOILBRENTEU ads_index        open        high         low  \
0      ...          NaN       NaN         NaN         NaN         NaN   
1      ...          NaN       NaN         NaN         NaN         NaN   
2      ...          NaN       NaN         NaN         NaN         NaN   
3      ...          NaN       NaN         NaN         NaN         NaN   
4      ...          NaN       NaN         NaN         NaN         NaN   
...    ...          ...       ...         ...         ...         ...   
62051  ...          NaN       NaN  234.449997  236.850006  234.449997   
62052  ...          NaN       NaN  233.889999  236.220001  232.600006   
62053  ...          NaN       NaN  234.080002  235.139999  227.759995   
62054  ...          NaN       NaN  229.979996  230.820007  228.410004   
62055  ...          NaN       NaN  229.740005  233.220001  229.570007   

            close      volume dividends stock_splits SP500  
0             NaN         NaN       NaN          NaN   NaN  
1             NaN         NaN       NaN          NaN   NaN  
2             NaN         NaN       NaN          NaN   NaN  
3             NaN         NaN       NaN          NaN   NaN  
4             NaN         NaN       NaN          NaN   NaN  
...           ...         ...       ...          ...   ...  
62051  236.479996  36254500.0       0.0          0.0   NaN  
62052  235.860001  38846600.0       0.0          0.0   NaN  
62053  230.759995  52287000.0       0.0          0.0   NaN  
62054  230.570007  31109500.0       0.0          0.0   NaN  
62055  231.410004  38776700.0       0.0          0.0   NaN  

[62056 rows x 43 columns]

In [6]:
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df.set_index('date', inplace=True)

In [7]:
# Step 2: Slice the data to include only dates after 2004-01-01
df = df.loc['2004-01-01':]

In [8]:
# List of columns to check for NaN values
columns_to_check = ['open', 'high', 'low', 'close', 'volume', 'SP500']

# Remove rows where all specified columns are NaN
df_cleaned = df.dropna(subset=columns_to_check, how='all')

In [9]:
df.describe()

Mkt-RF          SMB          HML          RMW          CMA  \
count  5201.000000  5201.000000  5201.000000  5201.000000  5201.000000   
mean      0.040711     0.000535    -0.001181     0.016435    -0.000094   
std       1.208168     0.631346     0.787480     0.453615     0.380189   
min     -12.000000    -4.550000    -4.970000    -2.720000    -2.740000   
25%      -0.430000    -0.360000    -0.330000    -0.230000    -0.200000   
50%       0.070000     0.000000    -0.020000     0.010000    -0.010000   
75%       0.590000     0.350000     0.310000     0.250000     0.190000   
max      11.350000     5.720000     6.730000     4.210000     2.520000   

                RF      USRECDP       USRECD      USRECDM    ads_index  \
count  5201.000000  7599.000000  7599.000000  7599.000000  7591.000000   
mean      0.005839     0.080011     0.080011     0.080274    -0.250928   
std       0.007202     0.271327     0.271327     0.271734     2.071278   
min       0.000000     0.000000     0.000000     0.000000   -26.524400   
25%       0.000000     0.000000     0.000000     0.000000    -0.372406   
50%       0.001000     0.000000     0.000000     0.000000    -0.095002   
75%       0.009000     0.000000     0.000000     0.000000     0.151029   
max       0.022000     1.000000     1.000000     1.000000     9.440280   

              open         high          low        close        volume  \
count  1258.000000  1258.000000  1258.000000  1258.000000  1.258000e+03   
mean    146.176032   147.800565   144.654762   146.308836  9.266660e+07   
std      41.786013    42.021225    41.525784    41.786841  5.291959e+07   
min      55.472059    55.574208    51.709551    54.569736  2.404830e+07   
25%     122.872769   124.133498   121.650088   123.003281  5.761422e+07   
50%     147.627858   149.251860   146.196051   147.771301  7.875405e+07   
75%     173.652050   175.598158   172.271674   173.622738  1.093357e+08   
max     236.206595   237.490005   234.449997   236.479996  4.265100e+08   

         dividends  stock_splits        SP500  
count  1258.000000   1258.000000  2512.000000  
mean      0.003549      0.003180  3280.603420  
std       0.028021      0.112777  1034.017927  
min       0.000000      0.000000  1829.080000  
25%       0.000000      0.000000  2381.642500  
50%       0.000000      0.000000  2951.235000  
75%       0.000000      0.000000  4155.500000  
max       0.250000      4.000000  5864.670000

In [10]:
df_cleaned.shape

(2517, 42)

In [11]:
# Drop columns that contain only NaN and 0.0 values
df = df.drop(columns=[col for col in df.columns if df[col].isna().all() or (df[col] == 0.0).all()])

In [12]:
df = df.apply(lambda col: pd.to_numeric(col.replace('.', np.nan), errors='coerce')).fillna(0.0)

In [13]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 7604 entries, 2004-01-01 to 2024-10-25
Data columns (total 42 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Mkt-RF          7604 non-null   float64
 1   SMB             7604 non-null   float64
 2   HML             7604 non-null   float64
 3   RMW             7604 non-null   float64
 4   CMA             7604 non-null   float64
 5   RF              7604 non-null   float64
 6   DGS10           7604 non-null   float64
 7   RIFSPPFAAD90NB  7604 non-null   float64
 8   DGS1            7604 non-null   float64
 9   CBETHUSD        7604 non-null   float64
 10  DBAA            7604 non-null   float64
 11  USRECDP         7604 non-null   float64
 12  USRECD          7604 non-null   float64
 13  DCPN3M          7604 non-null   float64
 14  DCPF1M          7604 non-null   float64
 15  DAAA            7604 non-null   float64
 16  T10Y3M          7604 non-null   float64
 17  BAMLH0A0HYM2   

In [14]:
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.expand_frame_repr', False)  # Prevent wrapping to a new line

In [15]:
# Drop the 'dividends' and 'stock_splits' columns
df = df.drop(columns=['dividends', 'stock_splits'])

In [16]:
# List of columns to check for NaN values
columns_to_check = ['open', 'high', 'low', 'close', 'volume', 'SP500']

# Remove rows where all specified columns are NaN
df_cleaned = df.dropna(subset=columns_to_check, how='all')

In [17]:
df_cleaned.shape

(7604, 40)

In [18]:
# Step 3: Remove columns with only 0.0 and NaN values
df_cleaned= df_cleaned.loc[:, ~(df_cleaned.isna().all() | (df_cleaned == 0.0).all())]

# Display the cleaned DataFrame
# print(df_cleaned)

In [19]:
# Check the data types of all columns
print(df_cleaned.dtypes)

Mkt-RF            float64
SMB               float64
HML               float64
RMW               float64
CMA               float64
RF                float64
DGS10             float64
RIFSPPFAAD90NB    float64
DGS1              float64
CBETHUSD          float64
DBAA              float64
USRECDP           float64
USRECD            float64
DCPN3M            float64
DCPF1M            float64
DAAA              float64
T10Y3M            float64
BAMLH0A0HYM2      float64
T10YIE            float64
DHHNGSP           float64
AMERIBOR          float64
USRECDM           float64
CBBTCUSD          float64
DEXUSUK           float64
DCOILWTICO        float64
VIXCLS            float64
DEXJPUS           float64
DEXUSEU           float64
T5YIE             float64
OBMMIJUMBO30YF    float64
NIKKEI225         float64
BAMLH0A0HYM2EY    float64
DCOILBRENTEU      float64
ads_index         float64
open              float64
high              float64
low               float64
close             float64
volume      

In [20]:
df_cleaned["tomorrow"] = df_cleaned["close"].shift(-1)

In [21]:
df_cleaned.shape

(7604, 41)

In [22]:
# List of columns to check for NaN values
columns_to_check = ['open', 'high', 'low', 'close', 'volume', 'SP500']
# Step 2: Remove rows where any of the specified columns are 0.0
df_cleaned = df_cleaned[~df_cleaned[columns_to_check].isin([0.0]).any(axis=1)]

# Display the cleaned DataFrame
print(df_cleaned)

            Mkt-RF   SMB   HML   RMW   CMA     RF  DGS10  RIFSPPFAAD90NB  DGS1  CBETHUSD  DBAA  USRECDP  USRECD  DCPN3M  DCPF1M  DAAA  T10Y3M  BAMLH0A0HYM2  T10YIE  DHHNGSP  AMERIBOR  USRECDM  CBBTCUSD  DEXUSUK  DCOILWTICO  VIXCLS  DEXJPUS  DEXUSEU  T5YIE  OBMMIJUMBO30YF  NIKKEI225  BAMLH0A0HYM2EY  DCOILBRENTEU  ads_index        open        high         low       close       volume    SP500    tomorrow
date                                                                                                                                                                                                                                                                                                                                                                                                                 
2019-10-28    0.62  0.28 -0.24  0.01 -0.04  0.007   1.85            1.77  1.60    181.87  4.01      0.0     0.0    1.78    0.00  3.08    0.20          3.85    1.64     2.49  1.860921      

In [23]:
df_cleaned.shape

(1253, 41)

In [24]:
df_cleaned['close_lag1'] = df_cleaned['close'].shift(1)
df_cleaned['close_lag2'] = df_cleaned['close'].shift(2)

In [25]:
y = df_cleaned['tomorrow']  # This is what you want to predict

# Define the features (X), including lagged close prices and other columns
X = df_cleaned[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF', 'DGS10', 'RIFSPPFAAD90NB',
         'DGS1', 'CBETHUSD', 'DBAA', 'USRECDP', 'USRECD', 'DCPN3M', 'DCPF1M',
         'DAAA', 'T10Y3M', 'BAMLH0A0HYM2', 'T10YIE', 'DHHNGSP', 'AMERIBOR',
         'USRECDM', 'CBBTCUSD', 'DEXUSUK', 'DCOILWTICO', 'VIXCLS', 'DEXJPUS',
         'DEXUSEU', 'T5YIE', 'OBMMIJUMBO30YF', 'NIKKEI225', 'BAMLH0A0HYM2EY',
         'DCOILBRENTEU', 'ads_index', 'open', 'high', 'low', 'volume', 'SP500',
         'close', 'close_lag1', 'close_lag2']]  

In [26]:
column_list = X.columns.tolist()
print(column_list)

['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF', 'DGS10', 'RIFSPPFAAD90NB', 'DGS1', 'CBETHUSD', 'DBAA', 'USRECDP', 'USRECD', 'DCPN3M', 'DCPF1M', 'DAAA', 'T10Y3M', 'BAMLH0A0HYM2', 'T10YIE', 'DHHNGSP', 'AMERIBOR', 'USRECDM', 'CBBTCUSD', 'DEXUSUK', 'DCOILWTICO', 'VIXCLS', 'DEXJPUS', 'DEXUSEU', 'T5YIE', 'OBMMIJUMBO30YF', 'NIKKEI225', 'BAMLH0A0HYM2EY', 'DCOILBRENTEU', 'ads_index', 'open', 'high', 'low', 'volume', 'SP500', 'close', 'close_lag1', 'close_lag2']


In [27]:
X[['close_lag1', 'close_lag2']] = X[['close_lag1', 'close_lag2']].fillna(0)

c:\Users\kandi\anaconda3\lib\site-packages\pandas\core\frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [28]:

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [29]:
X_scaled

array([[ 4.13825332e-01,  3.49967405e-01, -2.12624363e-01, ...,
        -2.06582897e+00, -3.50254592e+00, -3.48706887e+00],
       [-1.35558532e-01,  4.97159290e-01,  4.40816232e-01, ...,
        -2.09941186e+00, -2.05496758e+00, -3.48706887e+00],
       [ 1.57446196e-01, -7.78503716e-01, -1.02500997e+00, ...,
        -2.09958654e+00, -2.08844715e+00, -2.04405306e+00],
       ...,
       [-4.03319955e-02,  6.51967249e-03, -6.97683601e-04, ...,
         2.06829346e+00,  2.11617312e+00,  2.05286500e+00],
       [-4.03319955e-02,  6.51967249e-03, -6.97683601e-04, ...,
         2.07721077e+00,  2.06643701e+00,  2.11394086e+00],
       [-4.03319955e-02,  6.51967249e-03, -6.97683601e-04, ...,
         2.14589924e+00,  2.07532689e+00,  2.06436151e+00]])

In [30]:
correlation_matrix = np.abs(np.corrcoef(X_scaled.T, y.T))
correlation_with_target = correlation_matrix[-1][:-1]
selected_features = X.columns[correlation_with_target > 0.1]

In [31]:
selected_features.shape

(33,)

In [32]:
X_scaled.shape

(1253, 42)

In [33]:
## Kendalltau method for correlation 

In [34]:
## Creating a subset of the X columns for -- Apply Lagged Features and Rolling Statistics 

In [35]:
X1 = df_cleaned[['Mkt-RF', 'SP500', 'VIXCLS', 'DEXJPUS', 'NIKKEI225', 'T10Y3M', 'ads_index',
                'open', 'high', 'low', 'close']]

#### Step 1: Lagged Features and Rolling Statistics

In [36]:
# Initialize a new DataFrame to store the engineered features
X_features = X1.copy()

# Define the columns for which to create lagged features and rolling statistics
columns_to_lag = ['close', 'open', 'high', 'low']

# Add lagged features (1-day, 3-day, 5-day lags)
for column in columns_to_lag:
    X_features[f'{column}_lag1'] = X1[column].shift(1)
    X_features[f'{column}_lag3'] = X1[column].shift(3)
    X_features[f'{column}_lag5'] = X1[column].shift(5)

# Add rolling statistics (5-day and 10-day moving averages and volatility) for 'close'
X_features['close_ma5'] = X1['close'].rolling(window=5).mean()
X_features['close_ma10'] = X1['close'].rolling(window=10).mean()
X_features['close_vol5'] = X1['close'].rolling(window=5).std()
X_features['close_vol10'] = X1['close'].rolling(window=10).std()

# Drop rows with NaN values generated by lagging and rolling operations
X_features = X_features.dropna()
X_features.head()

Mkt-RF    SP500  VIXCLS  DEXJPUS  NIKKEI225  T10Y3M  ads_index       open       high        low      close  close_lag1  close_lag3  close_lag5  open_lag1  open_lag3  open_lag5  high_lag1  high_lag3  high_lag5   low_lag1   low_lag3   low_lag5  close_ma5  close_ma10  close_vol5  close_vol10
date                                                                                                                                                                                                                                                                                                         
2019-11-08    0.31  3093.08   12.07   109.14   23391.87    0.39  -0.254031  62.767832  63.192448  62.321381  63.119659   62.947388   62.202579   61.885677  62.779967  62.183222  60.366473  63.170617  62.459005  61.912284  62.627105  62.006632  60.274549  62.558178   61.280407    0.439387     1.612071
2019-11-11   -0.19  3087.01   12.69     0.00   23331.84    0.00  -0.176033  62.673212  63.685012  62.668362  63.619503   63.119659   62.229179   62.292084  62.767832  62.115481  62.250956  63.192448  62.289657  62.376754  62.321381  61.776807  61.779234  62.823662   61.617564    0.607338     1.721033
2019-11-12    0.16  3091.84   12.68   109.09   23520.01    0.33  -0.154580  63.461789  63.762665  63.308934  63.561272   63.619503   62.947388   62.202579  62.673212  62.779967  62.183222  63.685012  63.170617  62.459005  62.668362  62.627105  62.006632  63.095400   62.088239    0.562262     1.512387
2019-11-13    0.01  3094.04   13.00   108.83   23319.87    0.31  -0.135462  63.359873  64.245497  63.345315  64.170280   63.561272   63.119659   62.229179  63.461789  62.767832  62.115481  63.762665  63.192448  62.289657  63.308934  62.321381  61.776807  63.483620   62.620540    0.478542     1.134480
2019-11-14    0.07  3096.63   13.05   108.33   23141.55    0.25  -0.118672  63.995596  64.269778  63.595246  63.726273   64.170280   63.619503   62.947388  63.359873  62.673212  62.779967  64.245497  63.685012  63.170617  63.345315  62.668362  62.627105  63.639397   62.975389    0.376169     0.787385

Lagged Features
- **Purpose**: Lagged features capture the relationship between past values and future values of a stock’s price. Stock prices often follow certain patterns or trends, and past prices can be strong indicators of near-term movement.

- **Example**: If yesterday's (1-day lag) or last week’s price (5-day lag) had an upward trend, today’s price may follow that pattern.

- **Application**: In time series modeling, lagged values serve as predictors because they provide historical context, allowing the model to learn dependencies over time.

Rolling Statistics
- **Purpose**: **Rolling statistics**, like moving averages and volatility, help capture trends and momentum over a specified period. They smooth out short-term fluctuations, making it easier for the model to focus on underlying trends rather than noise.
- **Moving Average**: Shows the average stock price over recent days. For example, a 5-day moving average helps indicate if the stock is on an upward or downward trend.
- **Volatility**: Calculating the rolling standard deviation (volatility) indicates how much the price fluctuates over time, which can reflect market uncertainty or stability.

Lagged Features

- We create 1-day, 3-day, and 5-day lags for close, open, high, and low prices to capture recent trends.

Rolling Statistics

- We compute 5-day and 10-day moving averages (`close_ma5`, `close_ma10`) and rolling volatility (`close_vol5`, `close_vol10`) for the close price to track recent price trends and volatility.

Dropping NaN
- Since lagged and rolling features produce NaN values for the initial rows, we drop them to avoid issues during modeling.


#### Step 2: Feature Interactions

In [37]:
# Initialize a new DataFrame to store interaction features based on X_features
X_interactions = X_features.copy()

# Interaction between close price and SP500 (product and ratio)
X_interactions['close_SP500_product'] = X_features['close'] * X_features['SP500']
X_interactions['close_SP500_ratio'] = X_features['close'] / (X_features['SP500'] + 1e-5)  # Adding small value to avoid division by zero

# Interaction between close price and VIXCLS (volatility)
X_interactions['close_VIXCLS_product'] = X_features['close'] * X_features['VIXCLS']
X_interactions['close_VIXCLS_ratio'] = X_features['close'] / (X_features['VIXCLS'] + 1e-5)

# Interaction between SP500 and VIXCLS
X_interactions['SP500_VIXCLS_product'] = X_features['SP500'] * X_features['VIXCLS']
X_interactions['SP500_VIXCLS_ratio'] = X_features['SP500'] / (X_features['VIXCLS'] + 1e-5)

# Interaction between close and DEXJPUS (USD/JPY exchange rate)
X_interactions['close_DEXJPUS_product'] = X_features['close'] * X_features['DEXJPUS']

X_interactions.head()


Mkt-RF    SP500  VIXCLS  DEXJPUS  NIKKEI225  T10Y3M  ads_index       open       high        low      close  close_lag1  close_lag3  close_lag5  open_lag1  open_lag3  open_lag5  high_lag1  high_lag3  high_lag5   low_lag1   low_lag3   low_lag5  close_ma5  close_ma10  close_vol5  close_vol10  close_SP500_product  close_SP500_ratio  close_VIXCLS_product  close_VIXCLS_ratio  SP500_VIXCLS_product  SP500_VIXCLS_ratio  close_DEXJPUS_product
date                                                                                                                                                                                                                                                                                                                                                                                                                                                            
2019-11-08    0.31  3093.08   12.07   109.14   23391.87    0.39  -0.254031  62.767832  63.192448  62.321381  63.119659   62.947388   62.202579   61.885677  62.779967  62.183222  60.366473  63.170617  62.459005  61.912284  62.627105  62.006632  60.274549  62.558178   61.280407    0.439387     1.612071        195234.156171           0.020407            761.854289            5.229462            37333.4756          256.261594            6888.879630
2019-11-11   -0.19  3087.01   12.69     0.00   23331.84    0.00  -0.176033  62.673212  63.685012  62.668362  63.619503   63.119659   62.229179   62.292084  62.767832  62.115481  62.250956  63.192448  62.289657  62.376754  62.321381  61.776807  61.779234  62.823662   61.617564    0.607338     1.721033        196394.042022           0.020609            807.331493            5.013353            39174.1569          243.263008               0.000000
2019-11-12    0.16  3091.84   12.68   109.09   23520.01    0.33  -0.154580  63.461789  63.762665  63.308934  63.561272   63.619503   62.947388   62.202579  62.673212  62.779967  62.183222  63.685012  63.170617  62.459005  62.668362  62.627105  62.006632  63.095400   62.088239    0.562262     1.512387        196521.282192           0.020558            805.956925            5.012715            39204.5312          243.835770            6933.899126
2019-11-13    0.01  3094.04   13.00   108.83   23319.87    0.31  -0.135462  63.359873  64.245497  63.345315  64.170280   63.561272   63.119659   62.229179  63.461789  62.767832  62.115481  63.762665  63.192448  62.289657  63.308934  62.321381  61.776807  63.483620   62.620540    0.478542     1.134480        198545.414544           0.020740            834.213646            4.936172            40222.5200          238.002894            6983.651622
2019-11-14    0.07  3096.63   13.05   108.33   23141.55    0.25  -0.118672  63.995596  64.269778  63.595246  63.726273   64.170280   63.619503   62.947388  63.359873  62.673212  62.779967  64.245497  63.685012  63.170617  63.345315  62.668362  62.627105  63.639397   62.975389    0.376169     0.787385        197336.687469           0.020579            831.627857            4.883236            40411.0215          237.289473            6903.467109

- **SP500 Interaction Terms:** Interaction between close price and the SP500 index, as the general market trend often affects individual stocks.

- **Volatility Interaction Terms:** Interaction between close price and VIXCLS (volatility index) to capture how market uncertainty impacts Google’s stock.

- **Other Interactions:** Consider creating ratios and products with global indicators (e.g., **DEXJPUS** and **NIKKEI225**), especially if international markets influence Google’s valuation.


- **Market Index Interaction (SP500):**  The interaction between close price and the SP500 index can capture how closely Google’s price correlates with the overall market. Tech stocks, for example, often move in sync with the general market, and this interaction highlights that dependency.

- **Volatility Interaction (VIXCLS):** When the volatility index (VIXCLS) is high, it signals uncertainty in the market. The interaction between close price and VIXCLS can reflect how sensitive Google’s price is to market sentiment, which can be especially useful during periods of high or low market confidence.

- **Non-Linear Interactions:** Interactions introduce non-linearity by creating relationships that aren’t just additive but multiplicative. This is beneficial because stock price movements are rarely linear —they respond to compounded factors, such as a high market index coupled with low volatility, which often signals bullish conditions.


#### Step 3: Adding Technical Indicators

In [45]:
# Import necessary library
import pandas as pd

# Copy the current DataFrame to add new indicators
X_technical = X_interactions.copy()

# 1. Relative Strength Index (RSI) - 14-day RSI
window_length = 14
delta = X_technical['close'].diff()
gain = (delta.where(delta > 0, 0)).rolling(window=window_length).mean()
loss = (-delta.where(delta < 0, 0)).rolling(window=window_length).mean()
rs = gain / (loss + 1e-5)  # Adding small value to avoid division by zero
X_technical['RSI'] = 100 - (100 / (1 + rs))

# 2. Moving Average Convergence Divergence (MACD)
short_window = 12
long_window = 26
signal_window = 9
X_technical['MACD'] = X_technical['close'].ewm(span=short_window, adjust=False).mean() - X_technical['close'].ewm(span=long_window, adjust=False).mean()
X_technical['MACD_signal'] = X_technical['MACD'].ewm(span=signal_window, adjust=False).mean()

# 3. Bollinger Bands (20-day window, 2 standard deviations)
X_technical['MA20'] = X_technical['close'].rolling(window=20).mean()
X_technical['BB_upper'] = X_technical['MA20'] + (X_technical['close'].rolling(window=20).std() * 2)
X_technical['BB_lower'] = X_technical['MA20'] - (X_technical['close'].rolling(window=20).std() * 2)

# Drop NaN values introduced by rolling calculations
X_technical = X_technical.dropna()

X_technical.head()


Mkt-RF    SP500  VIXCLS  DEXJPUS  NIKKEI225  T10Y3M  ads_index       open       high        low      close  close_lag1  close_lag3  close_lag5  open_lag1  open_lag3  open_lag5  high_lag1  high_lag3  high_lag5   low_lag1   low_lag3   low_lag5  close_ma5  close_ma10  close_vol5  close_vol10  close_SP500_product  close_SP500_ratio  close_VIXCLS_product  close_VIXCLS_ratio  SP500_VIXCLS_product  SP500_VIXCLS_ratio  close_DEXJPUS_product        RSI      MACD  MACD_signal       MA20   BB_upper   BB_lower
date                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               
2019-12-06    0.91  3145.91   13.62   108.66   23354.40    0.31  -0.297809  64.900635  65.754716  64.856955  65.684349   64.439606   62.952248   64.844818  64.005290  62.675638  64.687105  64.514830  62.971655  65.026796  63.748095  62.185513  64.517256  64.135828   64.278745    1.035260     0.812363        206637.050552           0.020879            894.620834            4.822636            42847.2942          230.977070            7137.261369  56.774450  0.292794     0.241225  64.116056  65.502180  62.729931
2019-12-09   -0.33  3135.96   15.86   108.59   23430.70    0.29  -0.328144  65.512067  65.706174  64.277044  64.764748   65.684349   63.507870   64.095070  64.900635  63.345307  64.849668  65.754716  63.888812  65.087456  64.856955  63.250675  63.922799  64.269764   64.403459    1.071359     0.777513        203099.657945           0.020652           1027.168897            4.083525            49736.3256          197.727492            7032.803944  49.768437  0.310399     0.255060  64.198310  65.529637  62.866983
2019-12-10   -0.08  3132.52   15.68   108.73   23410.19    0.29  -0.336475  65.172380  65.529057  64.507549  65.143265   64.764748   64.439606   62.952248  65.512067  64.005290  62.675638  65.706174  64.514830  62.971655  64.277044  63.748095  62.185513  64.707967   64.454657    0.815212     0.810345        204062.579759           0.020796           1021.446392            4.154542            49117.9136          199.777934            7083.027178  52.755322  0.350850     0.274218  64.274498  65.640312  62.908684
2019-12-11    0.28  3141.63   14.99   108.67   23391.86    0.22  -0.344123  65.223324  65.778965  65.148107  65.698891   65.143265   65.684349   63.507870  65.172380  64.900635  63.345307  65.529057  65.754716  63.888812  64.507549  64.856955  63.250675  65.146172   64.611885    0.556745     0.888396        206401.605946           0.020912            984.826371            4.382845            47093.0337          209.581581            7139.498451  59.737086  0.422868     0.303948  64.381379  65.843361  62.919397
2019-12-12    0.90  3168.57   13.94   109.16   23424.81    0.34  -0.351091  64.973399  66.133205  64.861788  65.866302   65.698891   64.764748   64.439606  65.223324  65.512067  64.005290  65.778965  65.706174  64.514830  65.148107  64.277044  63.748095  65.431511   64.699717    0.461552     0.969432        208701.990081           0.020787            918.176257            4.724982            44169.8658          227.300411            7189.965580  62.294149  0.487828     0.340724  64.466180  66.066787  62.865574

- **Relative Strength Index (RSI)**: Measures the magnitude of recent price changes to evaluate overbought or oversold conditions.

- **Moving Average Convergence Divergence (MACD)**: Shows the relationship between two moving averages of the stock price, indicating trend changes.

- **Bollinger Bands**: Indicates volatility by placing bands around a moving average.


- **RSI**: A measure that ranges from 0 to 100, where values above 70 suggest overbought conditions, and values below 30 suggest oversold conditions.

- **MACD**: Calculated as the difference between a short-term (12-day) and long-term (26-day) exponential moving average (EMA). A signal line (9-day EMA of MACD) is used to identify buy/sell signals.

- **Bollinger Bands**: Consist of a 20-day moving average (center line) with upper and lower bands set at two standard deviations from the moving average, indicating volatility.

- **Technical Indicators**: Reveal hidden signals, like momentum shifts (MACD), overbought/oversold conditions (RSI), and volatility boundaries (Bollinger Bands). These insights can complement fundamental and market data to improve the model’s predictive power.


### Step 4: PCA Approach to handle the multi collinearity

In [42]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Standardize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_technical)

# Apply PCA
pca = PCA(n_components=0.95)  # Retain enough components to explain 95% of variance
X_pca = pca.fit_transform(X_scaled)

# Check the explained variance by each component
explained_variance = pca.explained_variance_ratio_
print("Explained variance by component:", explained_variance)
print("Total components selected:", pca.n_components_)


Explained variance by component: [0.67870932 0.1055555  0.0496881  0.03129894 0.02831451 0.024274
 0.01990889 0.01798534]
Total components selected: 8


- **PCA (Principal Component Analysis)**: Retains as much variance (information) as possible from the original correlated features by transforming them into a smaller set of uncorrelated components.

- **Purpose of PCA**: By using PCA, we retain the predictive signals from all the price-related and market features rather than removing them entirely.

- **Choosing n_components**: Setting PCA(n_components=0.95) will automatically capture 95% of the information in the dataset.

- **Explained Variance**: Review the explained_variance array to see how much each principal component contributes to the total variance.


In [44]:
X_scaled

array([[ 0.62105088, -1.46107128, -0.94400594, ..., -2.07498148,
        -2.19143898, -1.93544295],
       [-0.27812207, -1.4751827 , -0.67548855, ..., -2.0729116 ,
        -2.19076992, -1.93190532],
       [-0.0968372 , -1.48006141, -0.69706584, ..., -2.07099436,
        -2.18807303, -1.93082892],
       ...,
       [-0.03882604,  2.36327741, -0.2295579 , ...,  2.05104209,
         1.90742821,  2.18718167],
       [-0.03882604,  2.36185918, -0.2858986 , ...,  2.05516906,
         1.9182713 ,  2.18416214],
       [-0.03882604,  2.39476217, -0.41536234, ...,  2.06372497,
         1.94426797,  2.1741765 ]])